# Author: David Kocman, xkocma08

Include všech potřebných knihoven a načtení dat.

In [1]:
#!/usr/bin/python3.10
# coding=utf-8
import pandas as pd
from scipy.stats import chi2_contingency
from scipy.stats import chi2
from scipy.stats import mannwhitneyu

df = pd.read_pickle("accidents.pkl.gz")


## První úkol

Pomocí $\chi^{2}$ testu ověřit hypotézy s 95% jistotou (tedy $\alpha$ = 0.05) na základě dat nehodovosti. Nultá hypotéza zní následovně:

**H0: Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na silnicích třetí třídy.**

Jako první si vytáhneme data na základě zadání, a to tedy řádky které odpovídají fatální nehodě, která se stala na silnici 1. nebo 3. třídy.
S těmito daty vytvoříme *kontingenční tabulku*:

In [2]:
df_first = df.loc[(df["p13a"] > 0) & ((df["p36"] == 1) | (df["p36"] == 3)), ["p13a", "p36"]]

ct = pd.crosstab( index=df_first["p36"], columns=df_first["p13a"], values=df_first["p13a"], aggfunc="count")


Z knihovny *scipy.stats* použijeme funkce **chi2_contingency**, která vypočítá statistiku, pvalue, stupně volnosti, předpokládané frekvence a **chi2.ppf**, která vypočítá kritickou tabulkovou hodnotu.

Poté podle vypočítaných hodnot vyvodíme závěry.

In [3]:
# H0: Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako na silnicích třetí třídy.
stat, p, dof, expected = chi2_contingency(ct)

print(expected)
# vedly častěji

prob = 0.95
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
    print('Závislé (zamítáme H0)')
else:
    print('Nezávislé (nepodařilo se zamítnout H0)')

# Zamítli jsme H0

[[828.54746137  62.34216336  12.73657101   7.37380427]
 [407.45253863  30.65783664   6.26342899   3.62619573]]
probability=0.950, critical=7.815, stat=8.899
Závislé (zamítáme H0)



Můžeme na výsledku vidět, že realizace testové statistiky (*stat=8.899*) překročila kritickou hodnotu (*critical=7.815*), tudíž můžeme hypotézu **H0** zamítnout.

Z *expected* výstupu funkce můžeme vidět, že nehody na silnicích první třídy vedli častěji k fatálnímu výsledku nehody.

## Druhý úkol

V tomto úkolu budu ověřovat následující dvě hypotézy opět s 95% jistotou:

**H0: Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.**

Alternativní hypotéza:

**H1: Při nehodách vozidel značky Škoda je škoda na vozidle vyšší než při nehodách vozidel Audi.**

Pro tento test už ale nebudu užívat $\chi^{2}$ test, ale *Mann-Whitney U-Test*. Tento neparametrický test porovnává že dvě sady nepárových dat mají totéž rozdělení pravděpodobnosti . Opět si vytáhneme data na základě podmínek a rozdělíme je do dvou polí, jedno pro Škodu a jedno pro Audi. Tyto pole naplníme hodnotami ze sloupce *p53*, které obashují škody na jednotlivých vozech.

In [4]:
# 39 == skoda, 2 == audi
df_second = df.loc[((df["p45a"] == 39) | (df["p45a"] == 2)), ["p45a", "p53"]]

array_skoda = df_second[(df_second["p45a"] == 39)].p53.values

array_audi = df_second[(df_second["p45a"] == 2)].p53.values


Z toho důvodu, že test musí brát pole o stejných velikostech, rozhodl jsem se počítat pouze s prvními 1000 prvky v obou polích.

Použitá funkce ze *scipy.stats* je **mannwhitneyu**

In [5]:
# 0.05
res = mannwhitneyu(x=array_skoda[:1000], y=array_audi[:1000], alternative='greater')
print(res)

if res.pvalue < 0.05:
    print("větší škoda u značek škoda než u audi")
else:
    print("menší škoda u značek škoda než u audi")

#pvalue < 0.05 => výrazně nižší škoda na vozidle u značek Škoda než Audi

MannwhitneyuResult(statistic=438417.5, pvalue=0.9999991472891568)
menší škoda u značek škoda než u audi


Můžeme vidět, že *pvalue* (0.999999) nám vyšla velmi vysoká s *alternative=greater* a nenachází se na zvolené hladině významnosti (*pvalue > 0.05*), což znamená že příjímáme hypotézu *H0* a zamítáme alternativní *H1*.